# Flux sampling vs oxygen concentration 
In order to better see the shifts of flux within the metabolism we can use two different approaches first we can use flux variability analysis to determine the range of flux for each reaction. While this will show how each reaction is able to change it does not show in which direction. To be more accurate and have the ability to apply statistics to show change we can use flux sampling. We will be following the work done by [Hermann et al.](https://www.nature.com/articles/s41540-019-0109-0) and be able to ask some systems wide questions that we cannot with regular FBA. 

1) What is the flux distributions between Rnf and Fix under high and low oxygen concentrations?
2) How does carbon metabolism reorganize to increase energy to the ETS and nitrogenase? 
3) What is the metabolic cost of  increase oxygen on nitrogen fixation?

Three main comparisons in the samples are shifts of the ETS, shifts of the ED and TCA pathways, and shifts of nitrogenase flux. 

From the previous work done in ATPM_compare.ipynb we can show that one pathway NII_BD_R limits the excess ATPM in all conditions but we are still not accurate in the higher oxygen concentrations of 148 and 192 uM. And while a difference between Fix and Rnf is seen in the ATPM study the difference is only a small effect and can be studied in by sampling. So to be accurate we will be using the 108 uM O2 as our high oxygen and 12 uM o2 as our low with only the NDH II and cyt BD as the ETS path. Using 108 uM as not only because it is more accurate but because it also has similar growth yeilds as the higher O2 concentrations while still being distinct from 12 uM conditions. 

In [2]:
import cobra.test
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model
from cobra.sampling import sample
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from cobra.sampling import OptGPSampler
from scipy.stats import wilcoxon
from numpy import std, mean, sqrt
import seaborn as sns

In [3]:
#import model twice to make a high and low model

model_n2_highO2 = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")
model_n2_lowO2 = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")


In [4]:
#change bounds of model to match growth rate of ~0.2 at the experimental sucrose uptake and predicted ATPM while removing NDH I and Cytchrome C
#data 12uM NII_BD_R 3.79 suc 16.25 ATPM
#data 108uM NII_BD_R 9.32 suc 110.8 ATPM     #rounded 


model_n2_highO2.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_highO2.reactions.get_by_id("EX_glc__D_e").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("EX_glc__D_e").upper_bound = 0


model_n2_highO2.reactions.get_by_id("EX_sucr_e").lower_bound = -9
model_n2_highO2.reactions.get_by_id("EX_sucr_e").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("EX_sucr_e").lower_bound = -4
model_n2_lowO2.reactions.get_by_id("EX_sucr_e").upper_bound = 0

model_n2_highO2.reactions.get_by_id("PHBS_syn").upper_bound = 1000
model_n2_highO2.reactions.get_by_id("PHBS_syn").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("PHBS_syn").upper_bound = 1000
model_n2_lowO2.reactions.get_by_id("PHBS_syn").lower_bound = 0

model_n2_highO2.reactions.get_by_id("ATPM").lower_bound = 110
model_n2_highO2.reactions.get_by_id("ATPM").upper_bound = 110
model_n2_lowO2.reactions.get_by_id("ATPM").lower_bound = 16
model_n2_lowO2.reactions.get_by_id("ATPM").upper_bound = 16


model_n2_highO2.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_highO2.reactions.get_by_id("O2tpp").upper_bound = 1000
model_n2_lowO2.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("O2tpp").upper_bound = 1000

model_n2_highO2.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_highO2.reactions.get_by_id("NADH6").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("NADH6").upper_bound = 0

model_n2_highO2.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_highO2.reactions.get_by_id("CYOO2pp").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("CYOO2pp").upper_bound = 0

Lets run FBA on these conditions 


In [5]:
solution_model_n2_highO2 = model_n2_highO2.optimize()

u = solution_model_n2_highO2.objective_value
DT = 0.69314718056/u

#write flux to csv using pandas
df= pd.DataFrame.from_dict([solution_model_n2_highO2.fluxes]).T
df.to_csv('../Data/Sampling_O2/FBA_results/model_n2_highO2.csv')

#Output growth rate
print('High O2 sucrose growth rate: ' ,'%.3f' %u, ' 1/h' ' or a doubling time of ' ,'%.3f' %DT, 'hrs')
model_n2_highO2.summary()

High O2 sucrose growth rate:  0.201  1/h or a doubling time of  3.450 hrs


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001046,0,0.00%
cl_e,EX_cl_e,0.001046,0,0.00%
cobalt2_e,EX_cobalt2_e,5.023E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001425,0,0.00%
fe2_e,EX_fe2_e,0.001658,0,0.00%
fe3_e,EX_fe3_e,0.001569,0,0.00%
h_e,EX_h_e,0.2507,0,0.00%
k_e,EX_k_e,0.03922,0,0.00%
mg2_e,EX_mg2_e,0.001743,0,0.00%
mn2_e,EX_mn2_e,0.0001388,0,0.00%


In [6]:
solution_model_n2_highO2.fluxes["NIT1b"]

1.0506570392411834

In [7]:
import escher
from escher import Builder
from time import sleep

In [8]:
 Builder(model_json = "../Data/Models/iAA1300_C.json",
                 map_json = '../Data/Escher/Maps/Full_metabolism_AV.json',
                  reaction_data = solution_model_n2_highO2.fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 4.4808839231032875e-05, 'CPPPGO': 4.480883923103288e-05, 'GT…

In [9]:
solution_model_n2_lowO2 = model_n2_lowO2.optimize()

u = solution_model_n2_lowO2.objective_value
DT = 0.69314718056/u

#write flux to csv using pandas
df= pd.DataFrame.from_dict([solution_model_n2_lowO2.fluxes]).T
df.to_csv('../Data/Sampling_O2/FBA_results/model_n2_highO2.csv')

#Output growth rate
print('High O2 sucrose growth rate: ' ,'%.3f' %u, ' 1/h' ' or a doubling time of ' ,'%.3f' %DT, 'hrs')
model_n2_lowO2.summary()

High O2 sucrose growth rate:  0.219  1/h or a doubling time of  3.158 hrs


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001142,0,0.00%
cl_e,EX_cl_e,0.001142,0,0.00%
cobalt2_e,EX_cobalt2_e,5.487E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001556,0,0.00%
fe2_e,EX_fe2_e,0.001811,0,0.00%
fe3_e,EX_fe3_e,0.001714,0,0.00%
h_e,EX_h_e,0.2738,0,0.00%
k_e,EX_k_e,0.04284,0,0.00%
mg2_e,EX_mg2_e,0.001904,0,0.00%
mn2_e,EX_mn2_e,0.0001517,0,0.00%


from the cobra.sampling we will using the sample function to get the samples for each condition. As this is a very computaional process below will just be a demo at 10000 samples. Further samples require a computer cluster and can be found in the sampling.py script. 

In [10]:
S_model_n2_highO2 = sample(model_n2_highO2, n=10000, thinning=1000,  processes=8)

In [11]:
S_model_n2_highO2.to_csv("../Data/Sampling_O2/Sampling_results/model_n2_highO2_samples.csv.gz", compression="gzip")

In [12]:
S_model_n2_lowO2 = sample(model_n2_lowO2, n=10000, thinning=1000,  processes=8)

In [13]:
S_model_n2_lowO2.to_csv("../Data/Sampling_O2/Sampling_results/model_n2_lowO2_samples.csv.gz", compression="gzip")

In order to visualize some of the data we will use the histogram script from Herrmann et al as a template  

In [14]:
#reload tables not need if just ran above 

S_model_n2_highO2 = pd.read_csv("../Data/Sampling_O2/Sampling_results/model_n2_highO2_samples.csv.gz")
S_model_n2_lowO2= pd.read_csv("../Data/Sampling_O2/Sampling_results/model_n2_lowO2_samples.csv.gz")

In [15]:
# Load Function and modfied slightly from 
# https://github.com/HAHerrmann/FluxSamplingComparison/blob/master/ArabidopsisStudy/FluxSamplingAnalysisArabidopsis.ipynb
def make_svg(i,samples1,samples2,x1,x2,Condition):
    #print(i)
    x = samples1[i]
    y = samples2[i]
    bns = 100
    
    
    xn, xbins, xpatches = plt.hist(x, bins=bns, alpha=0.5,  color = "#b05abd", lw=0)
    yn, ybins, ypatches = plt.hist(y, bins=bns, alpha=0.5,  color = "#c7783b", lw=0)
    
    #plot FBA results as vertical line
    plt.axvline(x = x1,linewidth=2, linestyle='--', color="#b05abd")
    plt.axvline(x = x2,linewidth=2, linestyle='--', color="#c7783b")
    
    #determine if distrubtions are different
    stat, p = wilcoxon(x, y)
    print("Reaction {}".format(i), p)
    alpha = 0.05
    if p > alpha:
        print('Same distribution')
    else:
        print('Different distribution')
        
    plt.xticks([round(min(min(x),min(y)),3),round(max(max(x),max(y)),3)],fontsize=30)
    plt.yticks([],fontsize=30)
    plt.title("{}".format(i),y=0.99)
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_O2/{0:}/{0:}_{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 60, format="tiff")
    plt.close()

In [16]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", 
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    x1 = solution_model_n2_lowO2.fluxes[i]
    x2 = solution_model_n2_highO2.fluxes[i]
    make_svg(i,S_model_n2_lowO2,S_model_n2_highO2,x1,x2,"High_Low_O2")

Reaction NADH5 0.0
Different distribution
Reaction RNF 5.579652969011196e-15
Different distribution
Reaction FIX 0.0
Different distribution
Reaction NIT1b 0.0
Different distribution
Reaction CYTBDpp 0.0
Different distribution
Reaction ATPS4rpp 0.0
Different distribution
Reaction PDH 7.587803420435317e-14
Different distribution
Reaction GLNS 1.9113276063904888e-209
Different distribution
Reaction ICL 0.0
Different distribution
Reaction ICDHyr 0.0
Different distribution
Reaction NAD_H2 0.7474431553782116
Same distribution
Reaction HYD1pp 6.1212029170741075e-12
Different distribution
Reaction GLCDpp 0.0
Different distribution
Reaction EDD 0.0
Different distribution
Reaction HEX1 1.8463582989349723e-17
Different distribution
Reaction GND 0.0
Different distribution
Reaction EDD 0.0
Different distribution
Reaction CS 0.0
Different distribution
Reaction ICDHyr 0.0
Different distribution
Reaction AKGDH 0.0
Different distribution
Reaction GLUSy 6.349850162754404e-182
Different distribution
Reac

As there is some obivous  differences in the flux samples inbetween condtions most of the difference is due to flux rates. So we will normalize them to sucrose uptake rates in order to better compare the two.  

In [17]:
def make_svg_norm_suc(i,j,samples1,samples2,x1,x2,Condition):
    #"#b05abd" purple "#c7783b" orange
    x = samples1[i]
    y = samples2[i]
    #multiple to remove negative sign from sucrose import
    x_norm = (samples1[i]/(samples1[j]*-1))
    y_norm = (samples2[i]/(samples2[j]*-1))
    
    u1 = solution_model_n2_lowO2.fluxes[j]*-1
    u2 = solution_model_n2_highO2.fluxes[j]*-1
    
    bns = 100

    xn, xbins, xpatches = plt.hist(x_norm, bins=bns, alpha=0.5, color = "#b05abd", lw=0)
    yn, ybins, ypatches = plt.hist(y_norm, bins=bns, alpha=0.5,  color = "#c7783b", lw=0)
    
    plt.axvline(x = (x1/u1),linewidth=2, linestyle='--', color="#b05abd")
    plt.axvline(x = (x2/u2),linewidth=2, linestyle='--', color="#c7783b")
    

    #determine if distrubtions are different
    stat, p = wilcoxon(x, y)
    print("Reaction {}".format(i), p)
    alpha = 0.05
    if p > alpha:
        print('Same distribution')
    else:
        print('Different distribution')

    plt.xticks([round(min(min(x_norm),min(y_norm)),3),round(max(max(x_norm),max(y_norm)),3)],fontsize=30)
    plt.yticks([],fontsize=30)
    plt.title("{}".format(i),y=0.99, fontsize = 35)
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_O2/{0:}/{0:}_{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 60, format="tiff")
    plt.close()

In [18]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", 
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    x1 = solution_model_n2_lowO2.fluxes[i]
    x2 = solution_model_n2_highO2.fluxes[i]
    j = "EX_sucr_e"
    make_svg_norm_suc(i,j,S_model_n2_lowO2,S_model_n2_highO2,x1,x2,"High_Low_O2_norm_suc")

Reaction NADH5 0.0
Different distribution
Reaction RNF 5.579652969011196e-15
Different distribution
Reaction FIX 0.0
Different distribution
Reaction NIT1b 0.0
Different distribution
Reaction CYTBDpp 0.0
Different distribution
Reaction ATPS4rpp 0.0
Different distribution
Reaction PDH 7.587803420435317e-14
Different distribution
Reaction GLNS 1.9113276063904888e-209
Different distribution
Reaction ICL 0.0
Different distribution
Reaction ICDHyr 0.0
Different distribution
Reaction NAD_H2 0.7474431553782116
Same distribution
Reaction HYD1pp 6.1212029170741075e-12
Different distribution
Reaction GLCDpp 0.0
Different distribution
Reaction EDD 0.0
Different distribution
Reaction HEX1 1.8463582989349723e-17
Different distribution
Reaction GND 0.0
Different distribution
Reaction EDD 0.0
Different distribution
Reaction CS 0.0
Different distribution
Reaction ICDHyr 0.0
Different distribution
Reaction AKGDH 0.0
Different distribution
Reaction GLUSy 6.349850162754404e-182
Different distribution
Reac

In [19]:
def make_svg_bar_norm_suc(i,j,samples1,samples2,Condition):
    # "#bc5d41" orange #965da7 purplish

    
    #multiple to remove negative sign from sucrose import
    x_norm = (samples1[i]/(samples1[j]*-1))
    y_norm = (samples2[i]/(samples2[j]*-1))
   
    #round number to the 4 decimal some sampels only differ by 10^12 flux
    x_norm_r = x_norm.round(4)
    y_norm_r = y_norm.round(4)
    
    #create merged data frame
    df = pd.DataFrame()
   
    df["Low"] = x_norm_r
    df["High"] = y_norm_r
    
  
    #melt df 
    df2 = pd.melt(df)
    
    
    #measure effect size of data
    
    def cohen_d(x,y):
        nx = len(x)
        ny = len(y)
        dof = nx + ny - 2
        return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    d_xy = cohen_d(x_norm,y_norm)

    
    print("Reaction {} low vs high".format(i), d_xy)

    
    plt.figure(figsize=(2, 5))
    sns.boxplot(x="variable", y="value", data=df2, showfliers=False, width = 0.25, linewidth= 1.5)
    #plt.axhline(y = (x1/u1),linewidth=2, linestyle='--', color="#bc5d41")
    #plt.axhline(y = (x2/u2),linewidth=2, linestyle='--', color="#965da7")
    #plt.yticks([],fontsize=30)
    plt.title("{}".format(i),y=0.99, fontsize = 20)
    plt.xlabel('$[O_2]$')
    plt.ylabel('Relative Flux')
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_O2/{0:}/{0:}_{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 300, format="tiff")
    plt.close()

In [20]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    j = "EX_sucr_e"
    make_svg_bar_norm_suc(i,j,S_model_n2_lowO2,S_model_n2_highO2,"High_Low_O2_norm_suc_bar")

Reaction NADH5 low vs high -7.501004475335296
Reaction RNF low vs high 0.817085302822725
Reaction FIX low vs high 4.705706348764825
Reaction NIT1b low vs high 4.699957362375521
Reaction CYTBDpp low vs high -7.540212904096516
Reaction ATPS4rpp low vs high -9.296732447482814
Reaction PDH low vs high 0.8920958747087642
Reaction GLNS low vs high 4.657224107362482
Reaction ICL low vs high -4.316907823169067
Reaction ICDHyr low vs high -4.316907823169978
Reaction NAD_H2 low vs high -0.860074325986788
Reaction HYD1pp low vs high 1.1933982892829336
Reaction GLCDpp low vs high 0.7566533930665219
Reaction EDD low vs high -1.1253397225106399
Reaction HEX7 low vs high -0.9173589874217548
Reaction HEX1 low vs high 1.5193642723109715
Reaction GND low vs high -1.1253397225115436
Reaction EDD low vs high -1.1253397225106399
Reaction CS low vs high -4.284072060422106
Reaction ICDHyr low vs high -4.316907823169978
Reaction AKGDH low vs high -3.4992274032585953
Reaction GLUSy low vs high 4.14295761316601

In [21]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    j = "O2tpp"
    make_svg_bar_norm_suc(i,j,S_model_n2_lowO2,S_model_n2_highO2,"High_Low_O2_norm_O2_bar")

Reaction NADH5 low vs high 4.253068157141144
Reaction RNF low vs high -0.8648024977803909
Reaction FIX low vs high -4.843109711887524
Reaction NIT1b low vs high -4.801199936568114
Reaction CYTBDpp low vs high 2.1786895986155037
Reaction ATPS4rpp low vs high 5.965175107920816
Reaction PDH low vs high -0.9619676403414149
Reaction GLNS low vs high -4.974420711606844
Reaction ICL low vs high -0.8596427839709262
Reaction ICDHyr low vs high -0.859642783969134
Reaction NAD_H2 low vs high 0.9197211650769338
Reaction HYD1pp low vs high -1.264733128285718
Reaction GLCDpp low vs high -0.7562509293758806
Reaction EDD low vs high -3.727291933782115
Reaction HEX7 low vs high -6.972383345911725
Reaction HEX1 low vs high -1.6186277082478657
Reaction GND low vs high -3.7272919337813173
Reaction EDD low vs high -3.727291933782115
Reaction CS low vs high -0.9761475467967567
Reaction ICDHyr low vs high -0.859642783969134
Reaction AKGDH low vs high 1.2192937883118515
Reaction GLUSy low vs high -4.418133059

In [22]:
def make_svg_bar_norm_suc(i,j,k,samples1,samples2,Condition):
    # "#bc5d41" orange #965da7 purplish

    
    #multiple to remove negative sign from sucrose import
    x_norm = (samples1[i]/(samples1[j]*-1))
    y_norm = (samples2[i]/(samples2[j]*-1))
   
    #round number to the 4 decimal some sampels only differ by 10^12 flux
    x_norm_r = x_norm.round(4)
    y_norm_r = y_norm.round(4)
    
    #create merged data frame
    df = pd.DataFrame()
   
    df["Low"] = x_norm_r
    df["High"] = y_norm_r
    
  
    #melt df 
    df2 = pd.melt(df)
    
    
    #measure effect size of data
    
    def cohen_d(x,y):
        nx = len(x)
        ny = len(y)
        dof = nx + ny - 2
        return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    d_xy = cohen_d(x_norm,y_norm)

    
    print("Reaction {} low vs high".format(i), d_xy)

    
    plt.figure(figsize=(4, 1.5)) 
    
    sns.boxplot(y="variable", x="value", data=df2, showfliers=False, width = 0.25, linewidth= 1.5)
    sns.set_context("paper")
    #plt.axhline(y = (x1/u1),linewidth=2, linestyle='--', color="#bc5d41")
    #plt.axhline(y = (x2/u2),linewidth=2, linestyle='--', color="#965da7")
    plt.yticks([])
    plt.title("{}".format(k),y=1, fontsize = 12)
    plt.xlabel(' ', fontsize = 1)
    #plt.xlabel('Flux Relative to Sucrose Uptake ')
    plt.ylabel(' ', fontsize = 1)
    #plt.ylabel('$[O_2]$')
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_O2/{0:}/{0:}_{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 300, format="tiff")
    plt.close()

In [23]:
rxn = [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
          "GND", "CS",  "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1", "SUCDi"]
titles = [ "NDH I", "RNF", "FIX", "Nitrogenase", "Cytchrome bd", "ATP synthase",
         "Pyruvate dehydrogenase", "Glutamine synthetase", "Isocitrate lyase", "Isocitrate dehyrdogenase", "Soluble hydrogenase",
          "Uptake hydrogenase", "Glucose dehydrogenase", "6-P-gluconate dehydratase", "Hexokinase (fructose)",
          "Phosphogluconate dehydrogenase", "Citrate synthase", "2-Oxogluterate dehydrogenase", "Glutamate synthase", 
          "PEP carboxykinase", "Malate dehydrogenase", "Malic enzyme", "Succinate dehydrogenase"]
for i, k, in zip(rxn, titles):
    j = "EX_sucr_e"
    make_svg_bar_norm_suc(i,j,k,S_model_n2_lowO2,S_model_n2_highO2,"High_Low_O2_norm_O2_bar")

Reaction NADH5 low vs high -7.501004475335296
Reaction RNF low vs high 0.817085302822725
Reaction FIX low vs high 4.705706348764825
Reaction NIT1b low vs high 4.699957362375521
Reaction CYTBDpp low vs high -7.540212904096516
Reaction ATPS4rpp low vs high -9.296732447482814
Reaction PDH low vs high 0.8920958747087642
Reaction GLNS low vs high 4.657224107362482
Reaction ICL low vs high -4.316907823169067
Reaction ICDHyr low vs high -4.316907823169978
Reaction NAD_H2 low vs high -0.860074325986788
Reaction HYD1pp low vs high 1.1933982892829336
Reaction GLCDpp low vs high 0.7566533930665219
Reaction EDD low vs high -1.1253397225106399
Reaction HEX7 low vs high -0.9173589874217548
Reaction GND low vs high -1.1253397225115436
Reaction CS low vs high -4.284072060422106
Reaction AKGDH low vs high -3.4992274032585953
Reaction GLUSy low vs high 4.142957613166019
Reaction PPCK low vs high 0.8161005292909511
Reaction MDH2 low vs high -2.1228621638529783
Reaction ME1 low vs high 0.7477480418606173


In [24]:
print(max(S_model_n2_highO2["NIT1b"]))

0.7035933437663353


In [25]:
i = "NIT1b"
j = "EX_sucr_e"
k = "Nitrogenase1"
make_svg_bar_norm_suc(i, j, k,S_model_n2_lowO2,S_model_n2_highO2,"High_Low_O2_norm_O2_bar")

Reaction NIT1b low vs high 4.699957362375521


In [27]:
x_norm = (samples1["NIT1b"]/(samples1["EX_sucr_e"]*-1))
y_norm = (samples2["NIT1b"]/(samples2["EX_sucr_e"]*-1))

NameError: name 'samples1' is not defined